### Please run 'db_process.ipynb' file first. 

## CONFIGURATION

In [ ]:
import os
import json
import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")        

if openai_api_key is None:
    raise ValueError("{openai_api_key} is not set")

if openai_api_key == "":
    raise ValueError("{openai_api_key} is empty")

DB = "customer_relation.db"
MODEL = "gpt-4o-mini"
openai = OpenAI()

## TOOLS

In [ ]:
# Get the product price.
def get_price(product):
    """
    Get the product price.
    """
    try:
        print(f"Database Tool Called. Getting price for {product}", flush=True)
        with sqlite3.connect(DB) as conn:
            cursor = conn.cursor()
            search_term = f"%{product}%"
            cursor.execute("SELECT price FROM products WHERE product_name LIKE ?", (search_term,))
            result = cursor.fetchone()
            conn.commit()
            return f"The {product} price is ${result[0]}." if result else "The product not found."
    except Exception as e:
        return f"Error inserting to database: {str(e)}"


# describe the function.
price_function = {
    "name": "get_price",
    "description": "Get the price of the demanding prouduct.",
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The product that the customer is interested."
            },
        },
        "required": ["product_name"],
        "additioanlProperties": False
    }
}

In [ ]:
# Get the order status by this function.
def get_order_status(order):
    """
    Get the order status with order id.
    """
    try:
        print(f"Database Tool Called. Getting the order status for order {order}", flush=True)
        with sqlite3.connect(DB) as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT order_status FROM orders WHERE id=?", (order,))
            result = cursor.fetchone()
            conn.commit()
            return f"The order status is {result[0]}." if result else "The order not found."
    except Exception as e:
        return f"Error: {str(e)}"
        

# describe the function.
order_status_function = {
    "name": "get_order_status",
    "description": "Find out the current status of the customer's order.",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "integer",
                "description": "The order id of the customer's order."
            },
        },
        "required": ["order_id"],
        "additioanlProperties": False
    }
}

In [ ]:
# Take order.
def take_order(product_name, customer_id):
    """
    Receiving new order.
    """
    try:
        print(f"Database Tool Called. Taking order for {product_name} by customer {customer_id}", flush=True)
        with sqlite3.connect(DB) as conn:
            cursor = conn.cursor()
            search_term = f"%{product_name}%"
            row = cursor.execute("SELECT id FROM products WHERE product_name LIKE ?", (search_term,)).fetchone()
            if row is None:
                return f"The product '{product_name}' was not found."
            product_id = row[0]
            cursor.execute(
                "INSERT INTO orders(customer_id, product_id, order_status, order_date) VALUES(?, ?, ?, ?)",
                (customer_id, product_id, "pending", datetime.datetime.now()),
            )
            conn.commit()
            return f"Order with id {cursor.lastrowid} for {product_name} placed successfully."
    except Exception as e:
        return f"Error: {str(e)}"
        
take_order_function = {
    "name": "take_order",
    "description": "Take an order for a customer.",
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The product that the customer is interested."
            },
            "customer_id": {
                "type": "integer",
                "description": "The id of the customer."
            }
        },
        "required": ["product_name", "customer_id"],
        "additioanlProperties": False
    }
}

In [ ]:
# Save the customer to db.
def save_customer_data(first_name, last_name, phone, mail):
    """
    Save the customer data to db.
    """
    try:    
        print(f"Database Tool Called. Inserting customer {first_name} {last_name}.")
        # We pack them into a tuple here for the sqlite3 execute method
        customer_data = (first_name, last_name, phone, mail)
        with sqlite3.connect(DB) as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO customers(first_name, last_name, phone, mail) VALUES(?, ?, ?, ?)", customer_data)
            conn.commit()
            return f"Customer {first_name} {last_name} successfully inserted with ID {cursor.lastrowid}."
    except Exception as e:
        return f"Error inserting to database: {str(e)}"


save_customer_function = {
    "name": "save_customer_data",
    "description": "Saves a new customer's contact information to the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "first_name": {"type": "string", "description": "The customer's given name."},
            "last_name": {"type": "string", "description": "The customer's family name."},
            "phone": {"type": "string", "description": "The phone number including area code."},
            "mail": {"type": "string", "description": "The customer's email address."}
        },
        "required": ["first_name", "last_name", "phone", "mail"],
        "additioanlProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": order_status_function}, {"type": "function", "function": save_customer_function}, {"type": "function", "function": take_order_function}]
tools

## FUNCTIONS

In [ ]:
def handle_tool_call(message):
    """
    Handle the tool call.
    """
    responses = []
    for tool_call in message.tool_calls:
        tool = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        content = ""
        if tool == "get_price":
            content = get_price(arguments.get('product_name'))
        elif tool == "get_order_status":
            content = get_order_status(arguments.get('order_id'))
        elif tool == "take_order":
            content = take_order(**arguments)
        elif tool == "save_customer_data":
            content = save_customer_data(**arguments)

        responses.append({
            "role": "tool",
            "content": str(content),
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
system_message = """ 
    Role: You are a warm, efficient Sales Concierge for a boutique clothing store. Your goal is to provide a seamless shopping experience and guide customers toward a purchase.

    Product Catalog: You only sell: shirt, skirt, jeans, and socks.

    Operational Protocol:

        Proactive Assistance: Do not just wait for questions. Gently lead the conversation by asking for the customer's name/contact info (to "save their profile") or suggesting they place an order once a product is identified.

        Tool Usage: > * Use get_product_price for any pricing queries.

                      * Use save_customer_data early to personalize the session.

                      * Use take_order and get_order_status to manage the transaction lifecycle.

        Accuracy: Only provide information based on your tools and catalog. If a request is outside your scope, politely say you cannot assist with that.

    Style Constraints:

        Length: Maximum 3 sentences per response.

        Tone: Courteous, professional, and helpful.

        Focus: Keep the customer moving toward a completed order.
"""

In [ ]:
def chat(message, history):
    """
    Chat with the customer.
    """
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Customer Relation Chatbot",
    description="This is a chatbot for a boutique clothing store. It can answer questions, help customers find the products they need, and assist with placing orders.",
    textbox = gr.Textbox(placeholder="Enter your message here...", interactive=True),
    examples=[
        "I want to buy a shirt",
        "I want to know the price of the shirt",
        "I want to know the status of my order with id 1253",
        "I want to save my contact information",]
).launch()